# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

# 1st thing to construct writer agent to do the task
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of AI with DeepLearning.AI"

Are you eager to unlock the potential of artificial intelligence? Look no further than DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that dive deep into the realm of machine learning and AI. From neural networks to natural language processing, you can upskill and stay ahead in this rapidly evolving field. Whether you're a seasoned professional or just starting, DeepLearning.AI provides the tools and knowledge to propel your AI journey. Join the ranks of AI enthusiasts today and set off on a learning adventure with DeepLearning.AI!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
# create AssistantAgent from autogen, use LLM
# prompt critic to investigate writing of writer &provide feedback

critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
# initiate chat between 2 agents
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI with DeepLearning.AI"

Are you eager to unlock the potential of artificial intelligence? Look no further than DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that dive deep into the realm of machine learning and AI. From neural networks to natural language processing, you can upskill and stay ahead in this rapidly evolving field. Whether you're a seasoned professional or just starting, DeepLearning.AI provides the tools and knowledge to propel your AI journey. Join the ranks of AI enthusiasts today and set off on a learning adventure with DeepLearning.AI!

--------------------------------------------------------------------------------
Critic (to Writer):

## Nested chat

In [8]:
# create a bunch of reviewers, & meta reviewer to aggregate all reviews & give final suggestions

SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
# define task to be registered by sequential chat pattern
# between critic & reviewers; each chats have reviewer as recipient, critic is the sender by default

# get the content from the summary of auto chat session
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''



review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
    # use LLM reflection model to reflect the summary
     "summary_method": "reflection_with_llm",
    # summary prompt will be the review in json format
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
# register review chat, set the trigger by writer
# whenever critic receive message from writer, it roll message to nested chat for reflection
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
# same as the initiation in the top but with additional registration
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI with DeepLearning.AI"

Are you eager to unlock the potential of artificial intelligence? Look no further than DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that dive deep into the realm of machine learning and AI. From neural networks to natural language processing, you can upskill and stay ahead in this rapidly evolving field. Whether you're a seasoned professional or just starting, DeepLearning.AI provides the tools and knowledge to propel your AI journey. Join the ranks of AI enthusiasts today and set off on a learning adventure with DeepLearning.AI!

--------------------------------------------------------------------------------

******************

Critic (to Meta Reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 
{'Reviewer': 'SEO reviewer', 'Review': 'As an SEO reviewer, your content is engaging and informative, but it could benefit from some optimization. Here are some suggestions: 1. Include strategic keywords: Incorporate relevant keywords like "AI courses," "machine learning," and "Andrew Ng" to improve search engine visibility. 2. Optimize title tag: Revise the title to be more keyword-rich and concise, such as "DeepLearning.AI: AI Courses by Andrew Ng | Machine Learning Training." 3. Enhance meta description: Craft a compelling meta description highlighting key offerings like "neural networks" and "natural language processing" to entice users and improve click-through rates.'}
{'Reviewer': 'SEO reviewer', 'Review': 'As an SEO reviewer, your content is engaging and informative, but it could benefit from some optimization. Here are some suggestions: 1. Include strategic k

## Get the summary

In [15]:
print(res.summary)

Title: "Master AI and Machine Learning with DeepLearning.AI – Courses by Andrew Ng"

Unlock the potential of AI with DeepLearning.AI, Andrew Ng's platform offering top-tier AI courses in neural networks, natural language processing, and more. Whether you're a seasoned pro or just starting, these courses provide essential tools and knowledge for your AI journey. Explore cutting-edge AI concepts and gain valuable skills to stay ahead in this dynamic field. Join us today and embark on a learning adventure with DeepLearning.AI! Optimize your AI journey with expert-led courses tailored for success.

Meta Description: Dive into the world of AI and machine learning with DeepLearning.AI. Explore neural networks, NLP, and more in expert-led courses by Andrew Ng. Start your AI journey today!
